In [1]:
import requests
import pandas as pd
from datetime import date
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///COVID-Clinical-Trials.sqlite')
connection = engine.connect()
querylist = connection.execute("SELECT Count(nctid) FROM COVID_ClinicalTrials")
d, a = {}, []
for rowproxy in querylist:
    # rowproxy.items() returns an array like [(key0, value0), (key1, value1)] > thank you stackoverflow!
    for column, value in rowproxy.items():
        # build up the dictionary
        d = {**d, **{column: value}}
    a.append(d)
connection.close()
a
next_index = a[0]["Count(nctid)"]

In [3]:
next_index

153

In [4]:
today = date.today()
dateval = today.strftime("%m/%d/%Y")
min_rank = 1
max_rnk = 100
total = 2000
responses = []

# url = f"https://clinicaltrials.gov/api/query/full_studies?expr=COVID-19+AND+SEARCH%5BLocation%5D%28AREA%5BLocationCountry%5D+United+States%29&min_rnk={min_rank}&max_rnk={max_rnk}&fmt=json"
# response = requests.get(url)
# json = response.json()

try:
    while max_rnk < total:
        url = f"https://clinicaltrials.gov/api/query/full_studies?expr=COVID-19&min_rnk={min_rank}&max_rnk={max_rnk}&fmt=json"
        response = requests.get(url)
        json = response.json()
        for rank in json['FullStudiesResponse']['FullStudies']:
            responses.append(rank)
        min_rank += 100
        max_rnk += 100
except:
    pass




In [5]:
len(responses)

1765

In [6]:
responses[0]

{'Rank': 1,
 'Study': {'ProtocolSection': {'IdentificationModule': {'NCTId': 'NCT04395924',
    'OrgStudyIdInfo': {'OrgStudyId': 'CHRO-2020-10'},
    'Organization': {'OrgFullName': "Centre Hospitalier Régional d'Orléans",
     'OrgClass': 'OTHER'},
    'BriefTitle': 'Maternal-foetal Transmission of SARS-Cov-2',
    'OfficialTitle': 'Maternal-Foetal Transmission of COVID-19',
    'Acronym': 'TMF-COVID-19'},
   'StatusModule': {'StatusVerifiedDate': 'May 2020',
    'OverallStatus': 'Recruiting',
    'ExpandedAccessInfo': {'HasExpandedAccess': 'No'},
    'StartDateStruct': {'StartDate': 'May 5, 2020', 'StartDateType': 'Actual'},
    'PrimaryCompletionDateStruct': {'PrimaryCompletionDate': 'May 2021',
     'PrimaryCompletionDateType': 'Anticipated'},
    'CompletionDateStruct': {'CompletionDate': 'May 2021',
     'CompletionDateType': 'Anticipated'},
    'StudyFirstSubmitDate': 'May 19, 2020',
    'StudyFirstSubmitQCDate': 'May 19, 2020',
    'StudyFirstPostDateStruct': {'StudyFirstPostDa

In [7]:
n = 0
main_list = []
intervention_list = []
phase_list = []

for rank in responses:
    try:
        nctid = responses[n]["Study"]["ProtocolSection"]["IdentificationModule"]["NCTId"]
        brieftitle = responses[n]["Study"]["ProtocolSection"]["IdentificationModule"]["BriefTitle"]
        start_date = responses[n]["Study"]["ProtocolSection"]["StatusModule"]["StartDateStruct"]["StartDate"]
        completiondate = responses[n]["Study"]["ProtocolSection"]["StatusModule"]["CompletionDateStruct"]["CompletionDate"]
        overallstatus = responses[n]["Study"]["ProtocolSection"]["StatusModule"]["OverallStatus"]
        leadsponsor = responses[n]["Study"]["ProtocolSection"]["SponsorCollaboratorsModule"]["LeadSponsor"]["LeadSponsorName"]
        contactname = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["CentralContactList"]["CentralContact"][0]["CentralContactName"]
        contactphone = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["CentralContactList"]["CentralContact"][0]["CentralContactPhone"]
        contactemail = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["CentralContactList"]["CentralContact"][0]["CentralContactEMail"]
        briefsummary = responses[n]["Study"]["ProtocolSection"]["DescriptionModule"]["BriefSummary"]
        enrollmentcount = responses[n]["Study"]["ProtocolSection"]["DesignModule"]["EnrollmentInfo"]["EnrollmentCount"]
        primaryoutcomedesc = responses[n]["Study"]["ProtocolSection"]["OutcomesModule"]["PrimaryOutcomeList"]["PrimaryOutcome"][0]["PrimaryOutcomeMeasure"]
        facility = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationFacility"]
        locationcity = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationCity"]
        locationstate = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationState"]
        locationzip = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationZip"]
        locationcountry = responses[n]["Study"]["ProtocolSection"]["ContactsLocationsModule"]["LocationList"]["Location"][0]["LocationCountry"]
        main_dict = {"nctid":nctid,
               "brieftitle":brieftitle,
               "start_date":start_date,
               "completiondate":completiondate,
               "overallstatus":overallstatus,
               "leadsponsor":leadsponsor,
               "contactname":contactname,
               "contactphone": contactphone,
               "contactemail":contactemail,
               "briefsummary":briefsummary,
               "enrollmentcount":enrollmentcount,
               "primaryoutcomedesc":primaryoutcomedesc,
               "facility":facility,
               "locationcity":locationcity,
               "locationstate":locationstate,
               "locationzip":locationzip,
               "locationcountry":locationcountry,
               "statusDate":dateval}
        main_list.append(main_dict)
        for intervention in responses[n]["Study"]["ProtocolSection"]["ArmsInterventionsModule"]["InterventionList"]["Intervention"]:
            interventionname = intervention["InterventionName"]
            interventiontype = intervention["InterventionType"]
            interventiondesc = intervention["InterventionDescription"]
            intervention_dict = {
                "nctid": nctid,
                "interventionname": interventionname,
                "interventiontype": interventiontype,
                "interventiondesc": interventiondesc,
                "statusDate": dateval}
            intervention_list.append(intervention_dict)
        try:
            for phase in responses[n]["Study"]["ProtocolSection"]["DesignModule"]["PhaseList"]["Phase"]:
                phase_dict = {
                    "nctid": nctid,
                    "phase": phase,
                    "statusDate": dateval
                }
                phase_list.append(phase_dict)
        except:
            pass
        n+=1
    except:
        n+=1
        pass


In [8]:
trials_found = len(main_list)
trials_found

550

In [9]:
covid_df = pd.DataFrame(main_list)
intervention_df = pd.DataFrame(intervention_list)
phase_df = pd.DataFrame(phase_list)

In [10]:
covid_df.head()

,nctid,brieftitle,start_date,completiondate,overallstatus,leadsponsor,contactname,contactphone,contactemail,briefsummary,enrollmentcount,primaryoutcomedesc,facility,locationcity,locationstate,locationzip,locationcountry,statusDate
0,NCT04332380,Convalescent Plasma for Patients With COVID-19...,"May 1, 2020","December 31, 2020",Not yet recruiting,Universidad del Rosario,"Juan M Anaya Cabrera, MD, PhD",+57 321 233 9828,anayajm@gmail.com,Convalescent plasma (CP) has been used in rece...,10,Change in Viral Load,Universidad del Rosario,Bogota,Cundinamarca,11100,Colombia,05/26/2020
1,NCT04387929,Detection of Anti-COVID-19 Antibody Levels in ...,"May 4, 2020","May 30, 2021",Recruiting,Istituto Clinico Humanitas,"Patrizia Meroni, MD",02 82241,patrizia.meroni@humanitas.it,COVID-19 (SARS-CoV-2) infection in health prof...,6000,SARS-CoV-2 levels of IgG antibodies,Humanitas Rozzano/San Pio X,Rozzano,Lombardia,20089,Italy,05/26/2020
2,NCT04332835,Convalescent Plasma for Patients With COVID-19...,"May 1, 2020","December 31, 2020",Not yet recruiting,Universidad del Rosario,"Juan M Anaya Cabrera, MD, PhD",+57 321 233 9828,anayajm@gmail.com,Convalescent plasma (CP) has been used in rece...,80,Change in Viral Load,Universidad del Rosario,Bogota,Cundinamarca,11100,Colombia,05/26/2020
3,NCT04330261,Clinical Characteristics and Outcomes of Pedia...,"March 18, 2020","March 17, 2022",Recruiting,University of Calgary,"Stephen Freedman, MDCM, MSc",4039557740,stephen.freedman@ahs.ca,Rationale: The clinical manifestations of SARS...,12500,Clinical characteristics of children with SARS...,University of Calgary/Alberta Children's Hospital,Calgary,Alberta,T3B 6A8,Canada,05/26/2020
4,NCT04392323,Incidence of COVID-19 Test Conversion in Post-...,"May 13, 2020","July 31, 2020",Recruiting,Northwell Health,"Aaron Nizam, MD",9179570305,anizam1@northwell.edu,The current Sars-CoV-2 (COVID-19) pandemic has...,500,COVID-19 Test Conversion,North Shore University Hospital,Manhasset,New York,10030,United States,05/26/2020


In [11]:
phase_df.head()

,nctid,phase,statusDate
0,NCT04332380,Phase 2,05/26/2020
1,NCT04332835,Phase 2,05/26/2020
2,NCT04332835,Phase 3,05/26/2020
3,NCT04392323,Not Applicable,05/26/2020
4,NCT04341441,Phase 3,05/26/2020


In [12]:
intervention_df.head()

,nctid,interventionname,interventiontype,interventiondesc,statusDate
0,NCT04332380,Plasma,Drug,"Day 1: CP-COVID19, 250 milliliters. Day 2: CP-...",05/26/2020
1,NCT04387929,Detection of anti-COVID-19 antibody level,Diagnostic Test,Detection of anti-COVID-19 antibody level form...,05/26/2020
2,NCT04332835,Plasma,Drug,"Day 1: CP-COVID19, 250 milliliters. Day 2: CP-...",05/26/2020
3,NCT04332835,Hydroxychloroquine,Drug,400 milligrams each 12 hours for 10 days,05/26/2020
4,NCT04330261,Exposure (not intervention) - SARS-CoV-2 infec...,Other,Exposure is infection with the virus. There is...,05/26/2020


In [13]:
covid_df.describe()

,nctid,brieftitle,start_date,completiondate,overallstatus,leadsponsor,contactname,contactphone,contactemail,briefsummary,enrollmentcount,primaryoutcomedesc,facility,locationcity,locationstate,locationzip,locationcountry,statusDate
count,550,550,550,550,550,550,550,550,550,550,550,550,550,550,550,550,550,550
unique,550,550,124,220,2,378,510,501,502,550,169,527,446,262,183,370,38,1
top,NCT03261050,Use of the Interleukin-6 Inhibitor Clazakizuma...,May 2020,"December 31, 2020",Recruiting,Duke University,"Qing Ning, Professor",+8613971521450,qning@vip.sina.com,Background:\n\nThe SARS-CoV-2 virus has caused...,100,All-cause mortality,Massachusetts General Hospital,New York,California,10032,United States,05/26/2020
freq,1,1,40,24,412,9,5,5,5,1,36,5,6,29,42,12,286,550


In [14]:
#check if there are Null cells in covid_df
covid_df.isnull().values.any()

False

In [ ]:
# look at datatypes
# covid_df.dtypes

In [15]:
# remove day from start date column
covid_df['starting_date']=covid_df['start_date'].str.split(r'\s+').apply(lambda x: x[0:1] + x[2:] if len(x) > 2 else x).str.join(' ')
# covid_df
covid_df['completion_date']=covid_df['completiondate'].str.split(r'\s+').apply(lambda x: x[0:1] + x[2:] if len(x) > 2 else x).str.join(' ')

# drop initial date columns
covid_df = covid_df.drop(['start_date','completiondate' ], axis=1)
covid_df


,nctid,brieftitle,overallstatus,leadsponsor,contactname,contactphone,contactemail,briefsummary,enrollmentcount,primaryoutcomedesc,facility,locationcity,locationstate,locationzip,locationcountry,statusDate,starting_date,completion_date
0,NCT04332380,Convalescent Plasma for Patients With COVID-19...,Not yet recruiting,Universidad del Rosario,"Juan M Anaya Cabrera, MD, PhD",+57 321 233 9828,anayajm@gmail.com,Convalescent plasma (CP) has been used in rece...,10,Change in Viral Load,Universidad del Rosario,Bogota,Cundinamarca,11100,Colombia,05/26/2020,May 2020,December 2020
1,NCT04387929,Detection of Anti-COVID-19 Antibody Levels in ...,Recruiting,Istituto Clinico Humanitas,"Patrizia Meroni, MD",02 82241,patrizia.meroni@humanitas.it,COVID-19 (SARS-CoV-2) infection in health prof...,6000,SARS-CoV-2 levels of IgG antibodies,Humanitas Rozzano/San Pio X,Rozzano,Lombardia,20089,Italy,05/26/2020,May 2020,May 2021
2,NCT04332835,Convalescent Plasma for Patients With COVID-19...,Not yet recruiting,Universidad del Rosario,"Juan M Anaya Cabrera, MD, PhD",+57 321 233 9828,anayajm@gmail.com,Convalescent plasma (CP) has been used in rece...,80,Change in Viral Load,Universidad del Rosario,Bogota,Cundinamarca,11100,Colombia,05/26/2020,May 2020,December 2020
3,NCT04330261,Clinical Characteristics and Outcomes of Pedia...,Recruiting,University of Calgary,"Stephen Freedman, MDCM, MSc",4039557740,stephen.freedman@ahs.ca,Rationale: The clinical manifestations of SARS...,12500,Clinical characteristics of children with SARS...,University of Calgary/Alberta Children's Hospital,Calgary,Alberta,T3B 6A8,Canada,05/26/2020,March 2020,March 2022
4,NCT04392323,Incidence of COVID-19 Test Conversion in Post-...,Recruiting,Northwell Health,"Aaron Nizam, MD",9179570305,anizam1@northwell.edu,The current Sars-CoV-2 (COVID-19) pandemic has...,500,COVID-19 Test Conversion,North Shore University Hospital,Manhasset,New York,10030,United States,05/26/2020,May 2020,July 2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,NCT03620409,Identification of Cardiovascular and Molecular...,Recruiting,Jena University Hospital,"Sina M Coldewey, MD, PhD",+4936419323101,Sina.Coldewey@med.uni-jena.de,Sepsis is a life-threatening condition which c...,290,Mortality differences between septic patients ...,"NWG Translational Septomics, Zentrum für Innov...",Jena,Thuringia,D-07747,Germany,05/26/2020,May 2018,June 2021
546,NCT04390542,Intervention for Caregivers of Patients Underg...,Not yet recruiting,Case Comprehensive Cancer Center,"Sara Douglas, PhD, RN",1-800-641-2422,CTUReferral@UHhospitals.org,The purpose of this study is to determine whic...,40,Time to identify and recruit dyads in months,"University Hospitals Cleveland Medical Center,...",Cleveland,Ohio,44106,United States,05/26/2020,July 2020,February 2021
547,NCT04351880,Meals MATTER: A Trial of Medically Tailored Me...,Recruiting,Kaiser Permanente,"Rebecca S Boxer, MD, MS",303-636-2479,Rebecca.S.Boxer@kp.org,The purpose of this study is to determine if m...,640,Anxiety and Depression measured through the Ho...,St Joseph's Hospital,Denver,Colorado,80218,United States,05/26/2020,April 2020,July 2021
548,NCT03261050,Target Engagement of a Novel Dissonance-Based ...,Recruiting,Oregon Research Institute,"Eric Stice, PhD",541-484-2123,estice@ori.org,Most people with an eating disorder (ED) do no...,180,Change in occurrences of binge eating episodes...,Stanford University,Stanford,California,94305,United States,05/26/2020,July 2017,June 2022


In [16]:
engine = db.create_engine('sqlite:///COVID-Clinical-Trials.sqlite')

metadata = db.MetaData()

covid_df.to_sql("COVID_ClinicalTrials", con=engine, if_exists="append")
intervention_df.to_sql("Interventions", con=engine, if_exists="replace")
phase_df.to_sql("Phase_Recordings", con=engine, if_exists="append")

In [17]:
connection = engine.connect()
loclist = connection.execute("SELECT DISTINCT nctid, locationcity, locationstate, locationzip, locationcountry FROM COVID_ClinicalTrials")


In [18]:
loclist

In [19]:
d, a = {}, []
for rowproxy in loclist:
    # rowproxy.items() returns an array like [(key0, value0), (key1, value1)] > thank you stackoverflow!
    for column, value in rowproxy.items():
        # build up the dictionary
        d = {**d, **{column: value}}
    a.append(d)
connection.close()
a

[{'nctid': 'NCT04349202',
  'locationcity': 'Royal Oak',
  'locationstate': 'Michigan',
  'locationzip': '48073',
  'locationcountry': 'United States'},
 {'nctid': 'NCT04329832',
  'locationcity': 'Murray',
  'locationstate': 'Utah',
  'locationzip': '84107',
  'locationcountry': 'United States'},
 {'nctid': 'NCT04334382',
  'locationcity': 'Murray',
  'locationstate': 'Utah',
  'locationzip': '84107',
  'locationcountry': 'United States'},
 {'nctid': 'NCT04339998',
  'locationcity': 'Minneapolis',
  'locationstate': 'Minnesota',
  'locationzip': '55455',
  'locationcountry': 'United States'},
 {'nctid': 'NCT04331366',
  'locationcity': 'Atlanta',
  'locationstate': 'Georgia',
  'locationzip': '30322',
  'locationcountry': 'United States'},
 {'nctid': 'NCT04342663',
  'locationcity': 'Belleville',
  'locationstate': 'Illinois',
  'locationzip': '62220',
  'locationcountry': 'United States'},
 {'nctid': 'NCT04351243',
  'locationcity': 'New York',
  'locationstate': 'New York',
  'locat

In [21]:
a[0]["locationzip"]
a[0]["locationcity"]
querylist, mainziplist, nctidlist = [], [], []

for n in a:
    querylist.append(f"{n['locationcity']}, {n['locationstate']} {n['locationzip']} {n['locationcountry']}")
    mainziplist.append(n["locationzip"])
    nctidlist.append(n['nctid'])
querylist

['Royal Oak, Michigan 48073 United States',
 'Murray, Utah 84107 United States',
 'Murray, Utah 84107 United States',
 'Minneapolis, Minnesota 55455 United States',
 'Atlanta, Georgia 30322 United States',
 'Belleville, Illinois 62220 United States',
 'New York, New York 10029 United States',
 'Austin, Texas 78705 United States',
 'Durham, North Carolina 27710 United States',
 'Scottsdale, Arizona 85254 United States',
 'Hackensack, New Jersey 07601 United States',
 'New York, New York 10029 United States',
 'Honolulu, Hawaii 96813 United States',
 'San Francisco, California 94022 United States',
 'Chicago, Illinois 60452 United States',
 'Philadelphia, Pennsylvania 19107 United States',
 'Durham, North Carolina 27710 United States',
 'Chicago, Illinois 60637 United States',
 'Kansas City, Kansas 66160 United States',
 'Northglenn, Colorado 80260 United States',
 'Seattle, Washington 98195 United States',
 'Chicago, Illinois 60637 United States',
 'Aurora, Colorado 80045 United States'

In [22]:
from info import KEY
url = "http://open.mapquestapi.com/geocoding/v1/address?"
results = []

for n in querylist:
    query = f"{url}key={KEY}&location={n}"
    response = requests.get(query)
    json = response.json()
    resultDict = {}
    latitude = json["results"][0]["locations"][0]["latLng"]["lat"]
    longitude = json["results"][0]["locations"][0]["latLng"]["lng"]
    locationzip = mainziplist[querylist.index(n)]
    nctid = nctidlist[querylist.index(n)]
    resultDict = {
        "nctid":nctid,
        "locationzip":locationzip,
        "longitude":longitude,
        "latitude":latitude
    }
    results.append(resultDict)
results

[{'nctid': 'NCT04349202',
  'locationzip': '48073',
  'longitude': -83.144648,
  'latitude': 42.48948},
 {'nctid': 'NCT04329832',
  'locationzip': '84107',
  'longitude': -111.888417,
  'latitude': 40.666784},
 {'nctid': 'NCT04329832',
  'locationzip': '84107',
  'longitude': -111.888417,
  'latitude': 40.666784},
 {'nctid': 'NCT04339998',
  'locationzip': '55455',
  'longitude': -93.265469,
  'latitude': 44.9773},
 {'nctid': 'NCT04331366',
  'locationzip': '30322',
  'longitude': -84.390185,
  'latitude': 33.749099},
 {'nctid': 'NCT04342663',
  'locationzip': '62220',
  'longitude': -89.983993,
  'latitude': 38.52005},
 {'nctid': 'NCT04351243',
  'locationzip': '10029',
  'longitude': -73.986614,
  'latitude': 40.730646},
 {'nctid': 'NCT04352764',
  'locationzip': '78705',
  'longitude': -97.743699,
  'latitude': 30.271129},
 {'nctid': 'NCT04350073',
  'locationzip': '27710',
  'longitude': -78.901805,
  'latitude': 35.996653},
 {'nctid': 'NCT04344457',
  'locationzip': '85254',
  'lo

In [23]:
results_df = pd.DataFrame(results)
results_df.head()

,nctid,locationzip,longitude,latitude
0,NCT04349202,48073,-83.144648,42.489480
1,NCT04329832,84107,-111.888417,40.666784
2,NCT04329832,84107,-111.888417,40.666784
3,NCT04339998,55455,-93.265469,44.977300
4,NCT04331366,30322,-84.390185,33.749099


In [24]:
results_df.describe()

,longitude,latitude
count,573.000000,573.000000
mean,-49.501378,36.075910
std,70.546700,14.545181
min,-157.855676,-35.297591
25%,-100.445882,34.046399
50%,-77.094646,39.783730
75%,2.177432,41.569730
max,153.022791,60.167409


In [25]:
engine = db.create_engine('sqlite:///COVID-Clinical-Trials.sqlite')

metadata = db.MetaData()

results_df.to_sql("lat_long", con=engine, if_exists="replace")

In [26]:
connection = engine.connect()
query = connection.execute("SELECT DISTINCT c.nctid, c.brieftitle, c.facility, c.overallstatus, c.contactname, c.contactphone, c.contactemail, c.enrollmentcount, c.locationstate, l.latitude, l.longitude, p.phase FROM COVID_ClinicalTrials as c INNER JOIN lat_long as l on c.locationzip = l.locationzip INNER JOIN Phase_Recordings as p on c.nctid = p.nctid")


In [27]:
query

In [28]:
d, results = {}, []
for rowproxy in query:
    # rowproxy.items() returns an array like [(key0, value0), (key1, value1)] > thank you stackoverflow!
    for column, value in rowproxy.items():
        # build up the dictionary
        d = {**d, **{column: value}}
    results.append(d)
connection.close()
results

[{'nctid': 'NCT04329832',
  'brieftitle': 'Hydroxychloroquine vs. Azithromycin for Hospitalized Patients With Suspected or Confirmed COVID-19',
  'facility': 'Intermountain Medical Center',
  'overallstatus': 'Recruiting',
  'contactname': 'Valerie T Aston, MBA',
  'contactphone': '8015074606',
  'contactemail': 'Valerie.Aston@imail.org',
  'enrollmentcount': '300',
  'locationstate': 'Utah',
  'latitude': 40.666784,
  'longitude': -111.888417,
  'phase': 'Phase 2'},
 {'nctid': 'NCT04334382',
  'brieftitle': 'Hydroxychloroquine vs. Azithromycin for Outpatients in Utah With COVID-19',
  'facility': 'Intermountain Medical Center',
  'overallstatus': 'Recruiting',
  'contactname': 'Valerie T Aston, MBA',
  'contactphone': '8015074606',
  'contactemail': 'Valerie.Aston@imail.org',
  'enrollmentcount': '1550',
  'locationstate': 'Utah',
  'latitude': 40.666784,
  'longitude': -111.888417,
  'phase': 'Phase 3'},
 {'nctid': 'NCT04331366',
  'brieftitle': 'Bidirectional Oxygenation Valve in th